# AI Model Response Judging Workflow

This notebook serves as the main control panel for an end-to-end data processing and judging pipeline. It orchestrates the process from initial synthetic data generation to final analysis.

**Standard Operating Procedure:**
1. **Generate Synthetic Tasks**: Create `(input, context)` data pairs for a specified task type.
2. **Run Prompting Session**: Use the synthetic tasks to prompt a target model and generate a session log file.
3. **Reorganize Logs**: Restructure the flat log files into the nested directory format required by the ingestion script.
4. **Ingest Session Log**: Parse the structured log files and load the model responses into the database.
5. **Run Judging Process**: Find 'pending' items in the database and apply judgments.
6. **Analyze Results**: Query the database to view and analyze the final, judged data.

### 1. Environment Setup and Imports

This cell imports all necessary functions from the project's utility scripts and defines the constant file paths required for the workflow.

In [1]:
import sqlite3
import pandas as pd
import os
import shutil

# Import custom utility functions from project modules
from db_utils import create_connection, create_db_tables, clear_all_responses, get_record_count, get_status_breakdown
from data_generation import generate_spell_tasks, generate_element_editing_tasks, generate_automata_scripting_tasks
from prompting import run_prompting_session
from ingest_data import ingest_log_files
from run_judging import process_unjudged_instances

# Define project-wide directory and file constants
DB_FILE = "judgements.db"
SYNTHETIC_DATA_DIR = "synthetic_tasks"
SESSION_LOG_DIR = "session_logs"
PROMPT_PREAMBLE_DIR = "game_prompts"

# Ensure required directories exist
os.makedirs(SYNTHETIC_DATA_DIR, exist_ok=True)
os.makedirs(SESSION_LOG_DIR, exist_ok=True)
os.makedirs(PROMPT_PREAMBLE_DIR, exist_ok=True)

print("Setup complete. All modules imported and directories are ready.")

c:\Users\austi\OneDrive\Documents\MSc\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete. All modules imported and directories are ready.


### 2. (Optional) Full System Reset

This cell executes a destructive reset of the environment, which is useful for ensuring a clean test run. This operation will:
1. Delete all records from the database.
2. Erase all previously generated synthetic tasks and session logs.

In [2]:
print("--- Performing Full System Reset ---")
conn = create_connection(DB_FILE)
if conn:
    clear_all_responses(conn)
    create_db_tables(conn)
    conn.close()
    print("Database has been cleared and initialized.")

if os.path.exists(SYNTHETIC_DATA_DIR):
    shutil.rmtree(SYNTHETIC_DATA_DIR)
    os.makedirs(SYNTHETIC_DATA_DIR)
    print(f"Cleared directory: {SYNTHETIC_DATA_DIR}")

if os.path.exists(SESSION_LOG_DIR):
    shutil.rmtree(SESSION_LOG_DIR)
    os.makedirs(SESSION_LOG_DIR)
    print(f"Cleared directory: {SESSION_LOG_DIR}")
print("--- Reset Complete ---")

2025-06-16 12:25:06,197 - INFO - Successfully connected to SQLite database: judgements.db
2025-06-16 12:25:06,197 - WARNING - Clearing all records from the 'responses' table.
2025-06-16 12:25:06,203 - INFO - Table 'responses' has been cleared.
2025-06-16 12:25:06,204 - INFO - Creating 'responses' table if it doesn't exist...
2025-06-16 12:25:06,204 - INFO - Creating unique index 'idx_source' for data ingestion...
2025-06-16 12:25:06,205 - INFO - Creating index 'idx_problem_hash' for analysis...
2025-06-16 12:25:06,205 - INFO - Database tables and indexes are set up successfully.


--- Performing Full System Reset ---
Database has been cleared and initialized.
Cleared directory: synthetic_tasks
Cleared directory: session_logs
--- Reset Complete ---


### 3. Generate Synthetic Task Data

Execute a generator function to create a JSON file containing `(input, context)` pairs. The target task type for generation should be specified in the `TASK_TYPE_TO_GENERATE` variable.

In [3]:
# Specify the task type to generate data for.
TASK_TYPE_TO_GENERATE = "spellScripting"
SYNTHETIC_TASK_FILE = os.path.join(SYNTHETIC_DATA_DIR, f"{TASK_TYPE_TO_GENERATE}_tasks.json")

# Note: The GEMINI_API_KEY environment variable must be set for this step.
#generate_automata_scripting_tasks(num_tasks=5, output_file=SYNTHETIC_TASK_FILE)
generate_spell_tasks(num_tasks=5, output_file=SYNTHETIC_TASK_FILE)
#generate_element_editing_tasks(num_tasks=5, output_file=SYNTHETIC_TASK_FILE)


--- Starting Synthetic Spell Task Generation ---
Sending prompt to Gemini to generate 5 spell descriptions...
Successfully received 5 spell descriptions.

Successfully generated and saved 5 'spellScripting' tasks to 'synthetic_tasks\spellScripting_tasks.json'.


### 4. Define and Create Prompt Preamble

The `prompting.py` script requires a preamble file to define the core instruction for the model. This cell defines that preamble and writes it to a text file.

In [4]:

PREAMBLE_FILE = os.path.join(PROMPT_PREAMBLE_DIR, "spellScriptingOneShot.txt")

if False:
    PREAMBLE_FILE = os.path.join(PROMPT_PREAMBLE_DIR, "automata_preamble.txt")

    preamble_text = """You are an expert programmer for a 2D cellular automata simulation. 
    Your task is to translate a natural language description of a material's behavior into the required JSON format. 
    Analyze the user's input and the provided context of existing materials. 
    Return ONLY the JSON object for the new material, with no other text or explanation."""

    with open(PREAMBLE_FILE, 'w') as f:
        f.write(preamble_text)

    print(f"Preamble file created at: {PREAMBLE_FILE}")
else:
    with open(PREAMBLE_FILE, 'r') as f:
        preamble_text = f.read()
        print(f"Preamble file read from: {PREAMBLE_FILE}")

Preamble file read from: game_prompts\spellScriptingOneShot.txt


### 5. Execute Prompting Session

This step utilizes the generated synthetic data and preamble to prompt the target model, producing a session log file as output. This is the primary data collection step.

In [5]:
# Execute a prompting session by calling the main function from the 'prompting' script.
run_prompting_session(
    task_type=TASK_TYPE_TO_GENERATE,
    model_name='gpt-4.1', # This can be replaced with any other supported model.
    prompt_preamble=preamble_text,
    input_json_file=SYNTHETIC_TASK_FILE,
    output_log_dir=SESSION_LOG_DIR
)

# 'gemini-2.5-flash-preview-05-20' 'gpt-4.1'

--- Starting New Prompting Session ---
  Task Type: spellScripting
  Model: gpt-4.1
  Input File: synthetic_tasks\spellScripting_tasks.json
------------------------------------
Successfully loaded 5 tasks from 'synthetic_tasks\spellScripting_tasks.json'.
Will write results to: session_logs\LatentSpaceLog-gpt-4.1-2025-06-16-12-25-22.txt



2025-06-16 12:25:26,377 - INFO - HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


Processing task 1/5...


2025-06-16 12:25:30,427 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  -> Success: Received response from model.
Processing task 2/5...


...

  -> Success: Received response from model.
Processing task 4/5...


2025-06-16 12:25:38,762 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  -> Success: Received response from model.
Processing task 5/5...


2025-06-16 12:25:42,521 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  -> Success: Received response from model.

--- Prompting Session Complete ---
All 5 tasks have been processed and logged to session_logs\LatentSpaceLog-gpt-4.1-2025-06-16-12-25-22.txt.


### 6. Reorganize Log Files for Ingestion

The data ingestion script requires logs to be organized in a `group_name/session_name.txt` structure. This cell moves the flat log files into the required nested format. A `GROUP_LABEL` can be added to append a suffix to the model name, allowing for more descriptive grouping (e.g., to denote a specific prompt strategy).

In [6]:
print("--- Reorganizing log files for ingestion ---")

# Add a descriptive label to the model group name for this session.
# This is useful for differentiating runs with different prompt strategies, etc.
# Set to an empty string "" to use only the model name.
GROUP_LABEL = "test_group"

files_to_move = [f for f in os.listdir(SESSION_LOG_DIR) if os.path.isfile(os.path.join(SESSION_LOG_DIR, f))]

for filename in files_to_move:
    if filename.startswith("LatentSpaceLog-") and filename.endswith(".txt"):
        try:
            # Parse filename like: LatentSpaceLog-{model_name}-{timestamp}.txt
            base_name = filename.replace('LatentSpaceLog-', '').replace('.txt', '')
            parts = base_name.split('-')
            session_timestamp = '-'.join(parts[-6:])
            base_model_name = '-'.join(parts[:-6])
            
            # Append the label to the model name to create the final group name.
            final_group_name = f"{base_model_name}-{GROUP_LABEL}" if GROUP_LABEL else base_model_name

            # Create the necessary subdirectory for the final group name
            group_dir = os.path.join(SESSION_LOG_DIR, final_group_name)
            os.makedirs(group_dir, exist_ok=True)

            # Move the file into the subdirectory
            old_path = os.path.join(SESSION_LOG_DIR, filename)
            new_filename = f"Session-{session_timestamp}.txt"
            new_path = os.path.join(group_dir, new_filename)
            shutil.move(old_path, new_path)
            print(f"Moved {filename} -> {final_group_name}/{new_filename}")
        except IndexError:
            print(f"Could not parse filename '{filename}', skipping.")

print("--- Reorganization Complete ---")

--- Reorganizing log files for ingestion ---
Moved LatentSpaceLog-gpt-4.1-2025-06-16-12-25-22.txt -> gpt-4.1-test_group/Session-2025-06-16-12-25-22.txt
--- Reorganization Complete ---


### 7. Ingest Session Logs into Database

With the log files correctly structured, this step parses them and loads the data into the SQLite database. Ingested records are assigned a 'pending' status by default.

In [7]:
ingest_log_files(db_file=DB_FILE, master_folder=SESSION_LOG_DIR)

2025-06-16 11:34:43,759 - INFO - --- Starting Data Ingestion from 'session_logs' ---
2025-06-16 11:34:43,760 - INFO - Successfully connected to SQLite database: judgements.db
2025-06-16 11:34:43,772 - INFO - --- Data Ingestion Complete ---
2025-06-16 11:34:43,773 - INFO - Summary: 5 new records inserted, 0 records ignored as duplicates.


### 8. Verify Database Status

This cell provides a summary of the database's current state, verifying that records were ingested correctly before proceeding to the judging phase.

In [8]:
conn = create_connection(DB_FILE)
if conn:
    get_record_count(conn)
    get_status_breakdown(conn)
    conn.close()

2025-06-16 11:34:45,666 - INFO - Successfully connected to SQLite database: judgements.db
2025-06-16 11:34:45,667 - INFO - Total records in database: 5
2025-06-16 11:34:45,667 - INFO - Status breakdown:
2025-06-16 11:34:45,668 - INFO -   - Status: pending, Count: 5


### 9. Execute Judging Process

This script queries the database for all records with a 'pending' status. It then applies a judgment (currently simulated) and updates the record's status to 'judged'.

In [9]:
# The judging script requires a folder containing prompt components.
# For this workflow, a dummy directory is created to satisfy the script's requirement.
JUDGE_PROMPT_DIR = "judge_prompts"
#os.makedirs(JUDGE_PROMPT_DIR, exist_ok=True)

process_unjudged_instances(db_file=DB_FILE, prompt_folder=JUDGE_PROMPT_DIR, limit=10)

2025-06-16 11:34:49,152 - INFO - --- Starting Judging Process for up to 10 records ---
2025-06-16 11:34:49,153 - INFO - Successfully connected to SQLite database: judgements.db
2025-06-16 11:34:49,154 - INFO - Found 5 records to process.
2025-06-16 11:34:49,154 - INFO - Sending request to Gemini API...
2025-06-16 11:35:19,579 - INFO - Successfully updated record 1 with judging results.
2025-06-16 11:35:19,580 - INFO - Sending request to Gemini API...
2025-06-16 11:35:35,970 - INFO - Successfully updated record 2 with judging results.
2025-06-16 11:35:35,971 - INFO - Sending request to Gemini API...
2025-06-16 11:35:47,015 - INFO - Successfully updated record 3 with judging results.
2025-06-16 11:35:47,016 - INFO - Sending request to Gemini API...
2025-06-16 11:35:55,529 - INFO - Successfully updated record 4 with judging results.
2025-06-16 11:35:55,530 - INFO - Sending request to Gemini API...
2025-06-16 11:36:06,372 - INFO - Successfully updated record 5 with judging results.
2025-06

### 10. Final Analysis

The pipeline execution is complete. This final step retrieves the judged records from the database and displays them in a pandas DataFrame for analysis.

In [10]:
conn = create_connection(DB_FILE)
if conn:
    print("--- Final Database Status ---")
    get_status_breakdown(conn)
    
    print("\n--- Displaying Judged Records ---")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 80)
    df_final = pd.read_sql_query("SELECT * FROM responses WHERE status = 'judged'", conn)
    display(df_final.head())
    conn.close()

2025-06-16 11:36:14,023 - INFO - Successfully connected to SQLite database: judgements.db
2025-06-16 11:36:14,024 - INFO - Status breakdown:
2025-06-16 11:36:14,024 - INFO -   - Status: judged, Count: 5


--- Final Database Status ---

--- Displaying Judged Records ---


,row_id,problem_hash,group_name,session_name,task_key,input,context,model_response,status,judge_scores_json,judge_rationales_json,judged_at,ingested_at
0,1,89b8d82ec6a12a61357242899250092aff50200d97e22da32b46136534f134b5,gemini-2.5-flash-preview-05-20-test_group,Session-2025-06-16-11-33-38,spellScripting-2025-06-16-11-33-51,"**Chrono-Echo Step**: A ripple of displaced air, a faint shimmer, then the c...","[""Time"", ""Nature"", ""Arcane"", ""Aether"", ""Gravity""]","```json\n{\n""friendlyName"": ""Chrono Echo Step"",\n""components"": [\n{\n""compon...",judged,"{""adherence"": 1, ""correctness"": 2, ""fidelity"": 3, ""creativity"": 4, ""programm...","{""adherence"": ""The spell's root structure, including the 'friendlyName' and ...",2025-06-16T11:35:19.549997,2025-06-16T11:34:43.769649
1,2,d8ee0487b7ff1c10ac9adc7760ad839878e6006940f816ce0764fd40af7f673b,gemini-2.5-flash-preview-05-20-test_group,Session-2025-06-16-11-33-38,spellScripting-2025-06-16-11-34-14,"**Verdant Embrace**: From the caster's outstretched hand, a vibrant green en...","[""Nature"", ""Void"", ""Force"", ""Distortion"", ""Celestial"", ""Poison"", ""Mind"", ""Sp...","```json\n{\n""friendlyName"": ""Verdant Embrace"",\n""components"": [\n{\n""compone...",judged,"{""adherence"": 3, ""correctness"": 2, ""fidelity"": 5, ""creativity"": 5, ""programm...","{""adherence"": ""The spell script included a top-level 'friendlyName' key, whi...",2025-06-16T11:35:35.970216,2025-06-16T11:34:43.770550
2,3,89d8f8f6689ddf7d8375f210281bff6249a73ef5a738a9d350580db227fffef6,gemini-2.5-flash-preview-05-20-test_group,Session-2025-06-16-11-33-38,spellScripting-2025-06-16-11-34-21,**Whisperwind Ward**: The caster traces a symbol in the air; the air around ...,"[""Distortion"", ""Illusion"", ""Wood"", ""Mind"", ""Light"", ""Bone"", ""Gravity"", ""Sound""]","{""friendlyName"": ""Whisperwind Ward"", ""components"": [{""componentType"": ""shiel...",judged,"{""adherence"": 5, ""correctness"": 5, ""fidelity"": 5, ""creativity"": 5, ""programm...","{""adherence"": ""The model strictly adhered to the defined component types and...",2025-06-16T11:35:47.015328,2025-06-16T11:34:43.770571
3,4,ce9e8696a7b61813862be752025355e21d64f430c01d306a0985348b3700e5da,gemini-2.5-flash-preview-05-20-test_group,Session-2025-06-16-11-33-38,spellScripting-2025-06-16-11-34-29,"**Stone Meld**: Hands pressed against rock, the caster's skin momentarily ta...","[""Celestial"", ""Blood"", ""Rune"", ""Air"", ""Illusion"", ""Nether"", ""Order""]","```json\n{\n""friendlyName"": ""Stone Meld"",\n""components"": [\n{\n""componentTyp...",judged,"{""adherence"": 5, ""correctness"": 5, ""fidelity"": 4, ""creativity"": 3, ""programm...","{""adherence"": ""The model strictly used only the `componentType`s and their a...",2025-06-16T11:35:55.529528,2025-06-16T11:34:43.770578
4,5,6fed1616386b2e24fb03244141e681c2ac252fb8135bcd13f12a838bb1da43bb,gemini-2.5-flash-preview-05-20-test_group,Session-2025-06-16-11-33-38,spellScripting-2025-06-16-11-34-38,"**Ignis Nova**: A tiny spark ignites at the caster's fingertip, rapidly expa...","[""Order"", ""Ice"", ""Fire"", ""Force"", ""Mind"", ""Nature""]","{""friendlyName"": ""Ignis Nova"", ""components"": [{""componentType"": ""explosion"",...",judged,"{""adherence"": 5, ""correctness"": 5, ""fidelity"": 3, ""creativity"": 4, ""programm...","{""adherence"": ""The model strictly used only the defined component types and ...",2025-06-16T11:36:06.372692,2025-06-16T11:34:43.770584


In [11]:
import pandas as pd
import json

# Assume create_connection is defined in your db_utils.py
# from db_utils import create_connection

# --- Placeholder function to make the script runnable ---
import sqlite3
def create_connection(db_file):
    try:
        return sqlite3.connect(db_file)
    except sqlite3.Error as e:
        print(e)
    return None
# ---------------------------------------------------------


DB_FILE = "judgements.db"
conn = create_connection(DB_FILE)

if conn:
    print("--- Searching for records with failed programmatic validation ---")
    
    # This SQL query uses json_extract to look inside the JSON blob in the
    # judge_scores_json column and find records where the validation status is "Failed".
    sql_query = """
    SELECT
        row_id,
        model_response
    FROM
        responses
    WHERE
        status = 'judged' AND
        json_extract(judge_scores_json, '$.programmatic_validation') = 'Failed'
    """
    
    try:
        # Execute the query and load the results into a pandas DataFrame
        failed_df = pd.read_sql_query(sql_query, conn)
        
        if failed_df.empty:
            print("\n✅ No records with failed programmatic validation were found.")
        else:
            print(f"\nFound {len(failed_df)} records that failed programmatic validation:\n")
            
            # Iterate through the DataFrame and print each failed response
            for index, row in failed_df.iterrows():
                print(f"--- Failed Response (row_id: {row['row_id']}) ---")
                
                # Try to pretty-print the response if it's a JSON string
                try:
                    parsed_response = json.loads(row['model_response'])
                    print(json.dumps(parsed_response, indent=2))
                except (json.JSONDecodeError, TypeError):
                    # If it's not valid JSON or not a string, print as is
                    print(row['model_response'])
                
                print("-" * 35 + "\n")

    except Exception as e:
        print(f"An error occurred while querying the database: {e}")
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")

--- Searching for records with failed programmatic validation ---

Found 3 records that failed programmatic validation:

--- Failed Response (row_id: 1) ---
```json
{
"friendlyName": "Chrono Echo Step",
"components": [
{
"componentType": "explosion",
"radius": 64
},
{
"componentType": "element",
"element": "Time"
},
{
"componentType": "element",
"element": "Aether"
},
{
"componentType": "color",
"rgb": [
180,
190,
200
]
},
{
"componentType": "manaCost",
"cost": 35
},
{
"componentType": "deathTrigger",
"payload_components": [
{
"componentType": "teleportCaster"
},
{
"componentType": "manifestation",
"radius": 2,
"material_properties": {
"class": "gas",
"color_rgb": [
200,
210,
220
],
"blockpath": false,
"density": 0.5,
"elements": [
"Time",
"Aether"
],
"harmful": false,
"lifespan": 1.0
}
}
]
}
]
}
```
-----------------------------------

--- Failed Response (row_id: 2) ---
```json
{
"friendlyName": "Verdant Embrace",
"components": [
{
"componentType": "aoe",
"radius": 180,
"turns": 3
},